<a href="https://colab.research.google.com/github/Sebory/Sklearn/blob/main/Exercise01/exercise01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!git clone https://github.com/Sebory/Sklearn.git

fatal: destination path 'Sklearn' already exists and is not an empty directory.


In [30]:
class Review:
  def __init__(self, text, score):
    self.text = text
    self.score = score
    sentiment = get_sentiment()

  def get_sentiment(self):
    if self.score <= 2:
      return "NEGATIVE"
    elif self.score == 3:
      return "NEUTRAL"
    else: # score equal to 4 or 5
      return "POSITIVE"


    

In [31]:
import json

file_name = "/content/Sklearn/Exercise01/Books_small.json"

reviews = []
with open(file_name) as f:
  for line in f:
    review = json.loads(line)
    reviews.append(Review(review["reviewText"], review["overall"]))

reviews[5].text
  

TypeError: ignored